# Setup inicial

Nesse trecho preparamos a máquina virtual no Google Colab para executar o ambiente do pêndulo invertido com ações contínuas, mostrando o resultado como um vídeo

In [1]:
#@title
%%capture
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1

import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor
gymlogger.set_level(40) #error only
import tensorflow as tf
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import base64
from IPython.display import HTML

from IPython import display as ipythondisplay

from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

"""
Utility functions to enable video recording of gym environment and displaying it
To enable video, just do "env = wrap_env(env)""
"""

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    

def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

"""
Classic cart-pole system implemented by Rich Sutton et al.
Copied from http://incompleteideas.net/sutton/book/code/pole.c
permalink: https://perma.cc/C9ZM-652R

Continuous version by Ian Danforth
"""

import math
import gym
from gym import spaces, logger
from gym.utils import seeding
import numpy as np


class ContinuousCartPoleEnv(gym.Env):
    metadata = {
        'render.modes': ['human', 'rgb_array'],
        'video.frames_per_second': 50
    }

    def __init__(self):
        self.gravity = 9.8
        self.masscart = 1.0
        self.masspole = 0.1
        self.total_mass = (self.masspole + self.masscart)
        self.length = 0.5  # actually half the pole's length
        self.polemass_length = (self.masspole * self.length)
        self.force_mag = 30.0
        self.tau = 0.02  # seconds between state updates
        self.min_action = -1.0
        self.max_action = 1.0

        # Angle at which to fail the episode
        self.theta_threshold_radians = 12 * 2 * math.pi / 360
        self.x_threshold = 2.4

        # Angle limit set to 2 * theta_threshold_radians so failing observation
        # is still within bounds
        high = np.array([
            self.x_threshold * 2,
            np.finfo(np.float32).max,
            self.theta_threshold_radians * 2,
            np.finfo(np.float32).max])

        self.action_space = spaces.Box(
            low=self.min_action,
            high=self.max_action,
            shape=(1,)
        )
        self.observation_space = spaces.Box(-high, high)

        self.seed()
        self.viewer = None
        self.state = None

        self.steps_beyond_done = None

    def seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

    def stepPhysics(self, force):
        x, x_dot, theta, theta_dot = self.state
        costheta = math.cos(theta)
        sintheta = math.sin(theta)
        temp = (force + self.polemass_length * theta_dot * theta_dot * sintheta) / self.total_mass
        thetaacc = (self.gravity * sintheta - costheta * temp) / \
            (self.length * (4.0/3.0 - self.masspole * costheta * costheta / self.total_mass))
        xacc = temp - self.polemass_length * thetaacc * costheta / self.total_mass
        x = x + self.tau * x_dot
        x_dot = x_dot + self.tau * xacc
        theta = theta + self.tau * theta_dot
        theta_dot = theta_dot + self.tau * thetaacc
        return (x, x_dot, theta, theta_dot)

    def step(self, action):
        assert self.action_space.contains(action), \
            "%r (%s) invalid" % (action, type(action))
        # Cast action to float to strip np trappings
        force = self.force_mag * float(action)
        self.state = self.stepPhysics(force)
        x, x_dot, theta, theta_dot = self.state
        done = x < -self.x_threshold \
            or x > self.x_threshold \
            or theta < -self.theta_threshold_radians \
            or theta > self.theta_threshold_radians
        done = bool(done)

        if not done:
            reward = 1.0
        elif self.steps_beyond_done is None:
            # Pole just fell!
            self.steps_beyond_done = 0
            reward = 1.0
        else:
            if self.steps_beyond_done == 0:
                logger.warn("""
You are calling 'step()' even though this environment has already returned
done = True. You should always call 'reset()' once you receive 'done = True'
Any further steps are undefined behavior.
                """)
            self.steps_beyond_done += 1
            reward = 0.0

        return np.array(self.state), reward, done, {}

    def reset(self):
        self.state = self.np_random.uniform(low=-0.05, high=0.05, size=(4,))
        self.steps_beyond_done = None
        return np.array(self.state)

    def render(self, mode='human'):
        screen_width = 600
        screen_height = 400

        world_width = self.x_threshold * 2
        scale = screen_width /world_width
        carty = 100  # TOP OF CART
        polewidth = 10.0
        polelen = scale * 1.0
        cartwidth = 50.0
        cartheight = 30.0

        if self.viewer is None:
            from gym.envs.classic_control import rendering
            self.viewer = rendering.Viewer(screen_width, screen_height)
            l, r, t, b = -cartwidth / 2, cartwidth / 2, cartheight / 2, -cartheight / 2
            axleoffset = cartheight / 4.0
            cart = rendering.FilledPolygon([(l, b), (l, t), (r, t), (r, b)])
            self.carttrans = rendering.Transform()
            cart.add_attr(self.carttrans)
            self.viewer.add_geom(cart)
            l, r, t, b = -polewidth / 2, polewidth / 2, polelen-polewidth / 2, -polewidth / 2
            pole = rendering.FilledPolygon([(l, b), (l, t), (r, t), (r, b)])
            pole.set_color(.8, .6, .4)
            self.poletrans = rendering.Transform(translation=(0, axleoffset))
            pole.add_attr(self.poletrans)
            pole.add_attr(self.carttrans)
            self.viewer.add_geom(pole)
            self.axle = rendering.make_circle(polewidth / 2)
            self.axle.add_attr(self.poletrans)
            self.axle.add_attr(self.carttrans)
            self.axle.set_color(.5, .5, .8)
            self.viewer.add_geom(self.axle)
            self.track = rendering.Line((0, carty), (screen_width, carty))
            self.track.set_color(0, 0, 0)
            self.viewer.add_geom(self.track)

        if self.state is None:
            return None

        x = self.state
        cartx = x[0] * scale + screen_width / 2.0  # MIDDLE OF CART
        self.carttrans.set_translation(cartx, carty)
        self.poletrans.set_rotation(-x[2])

        return self.viewer.render(return_rgb_array=(mode == 'rgb_array'))

    def close(self):
        if self.viewer:
            self.viewer.close()

# Inspecionando as entradas e saídas

In [2]:
# Cria o ambiente do pêndulo invertido contínuo
env = wrap_env(ContinuousCartPoleEnv())

In [3]:
# Examina as entradas e saídas
print('Estados:')
print(env.observation_space.low)
print(env.observation_space.high)
print('Ações:')
print(env.action_space.low)
print(env.action_space.high)

Estados:
[-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38]
[4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38]
Ações:
[-1.]
[1.]


# Fuzzy

In [4]:
class Fuzzy():
  def __init__(self, a, b, c):
    self.a = a
    self.b = b
    self.c = c
    self.fz = np.array([0, 0, 1, 0, 0])
  
  def fuzzify(self, x):
    
    #Neutro
    if x < -self.a or x > self.a:
      n = 0
    
    else:
      n = 1 - abs(x) / self.a
    
    #Baixo positivo
    if x < 0 or x > self.b:
      bp = 0
    
    else:
      bp = 1 - abs(x - self.a) / (self.b - self.a)
    
    #Alto positivo
    if x < self.a:
      ap = 0
    
    elif x > self.b:
      ap = 1
    
    else:
      ap = (x-self.a)/(self.b - self.a)
    
    #Baixo negativo
    if x > 0 or x < -self.b:
      bn = 0
    
    else:
      bn = 1 - abs(x+self.a) / (self.b - self.a)
    
    #Alto negativo
    if x > -self.a:
      an = 0
    elif x < -self.b:
      an = 1
    else:
      an = 1 - (x-self.b) / (self.b - self.a)

    self.fz = np.array([an, bn, n, bp, ap])
  
  def get(self):
    return self.fz
  
  def set(self, an, bn, n, bp, ap):
    self.fz = np.array([an, bn, n, bp, ap])
  
  def defuzzify(self):
    x_an = -self.c
    x_bn = -self.a
    x_n = 0
    x_bp = self.a
    x_ap = self.c
    an, bn, n, bp, ap = self.fz
    s = an + bn + n + bp + ap
    x = (x_an*an + x_bn*bn + x_n*n + x_bp*bp + x_ap*ap) / s
    return x

In [6]:
class FuzzyController():
  def __init__(self):
    self.ang = Fuzzy(0.05, 0.2, 0.5)
    self.velang = Fuzzy(0.5, 2.0, 5.0)
    self.action = Fuzzy(0.1, 0.5, 1.0)
  
  def compute(self, ang, velang):
    self.ang.fuzzify(ang)
    self.velang.fuzzify(velang)
    aan, abn, an, abp, aap = self.ang.get()
    van, vbn, vn, vbp, vap = self.velang.get()
    ap = max(min(an, van), min(aan, vn))
    bp = max(min(an, vbn), min(abn, vn))
    n = max(min(abp, vbn), min(an, vn), min(abn, vbp))
    bn = max(min(an, vbp), min(abp, vn))
    an = max(min(an, vap), min(aap, vn))
    
    s = an+bn+n+bp+ap

    if s == 0:
      self.action.set(0, 0, 1, 0, 0)
    else:
      self.action.set(an/s, bn/s, n/s, bp/s, ap/s)
      
    return self.action.defuzzify()

# Rodando o ambiente

In [7]:
# Cria o ambiente
env = wrap_env(ContinuousCartPoleEnv())

# Cria controlador Fuzzy
fc = FuzzyController()

# Laço principal
done = False
observation = env.reset()
R = 0.0
while not done:
  env.render()
  pos, vel, ang, velang = observation
  action = np.array([-fc.compute(ang, velang)])
  # action = env.action_space.sample()
  observation, reward, done, info = env.step(action)
  R += reward

# Encerra e mostra vídeo do resultado
env.close()
show_video()
print('Total reward:', R)

Total reward: 8308.0
